# Guest lecture Working with text: Topic Modelling
## by Damian Trilling

**Amsterdam, 18-1-2018**

In this tutorial, I showcase how you can use some automated content analysis techniques (see, e.g., Boumans & Trilling, 2016) using Python and Jupter Notebook.

Boumans, J. W., & Trilling, D. (2016). Taking stock of the toolkit: An overview of relevant automated content analysis approaches and techniques for digital journalism scholars. Digital Journalism, 4(1), 8–23. doi:doi.org/10.1080/21670811.2015.1096598

## Further ressources:
- More slides: https://github.com/damian0604/bdaca/blob/master/courses/turku2017/lecture.pdf
- The complete ACA course book: https://github.com/damian0604/bdaca/blob/master/book/bd-aca_book.pdf
- Other Notebooks: https://github.com/damian0604/bdaca/tree/master/ipynb
- in particular, a longer and more elaborated version of this one: https://github.com/damian0604/bdaca/blob/master/ipynb/ic2s2.ipynb


NLTK is the Natural Language Toolkit. We need to download some ressources it uses, like lists of stopwords or a POS-tagger.

In [ ]:
import nltk
# nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

We also need to install some additional packages:

In [ ]:
!pip install --user gensim
!pip install --user pyLDAvis

You probably have to restart your kernel now.

# Let's get started!

## Import modules
Before we start, let's import some modules that we need today. It is good practice to do so at the beginning of a script, so we'll do it right now and not later when we need them. The benefit is that you immediately see if something goes wrong (for instance, because the module is not installed).

In [11]:
import csv
import re
from glob import glob
from string import punctuation
import random
import os
random.seed("ic2s2colgne")
from nltk.sentiment import vader
from nltk.corpus import stopwords
import nltk
from gensim import corpora
from gensim import models
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
print("STARTING NOW:",str(datetime.now()))

STARTING NOW: 2018-01-21 17:59:55.173807


In [2]:
# get more output when estimating topic models:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Download the data
We will use a dataset by Schumacher et al. (2016). From the abstract:
> This paper presents EUSpeech, a new dataset of 18,403 speeches from EU leaders (i.e., heads of government in 10 member states, EU commissioners, party leaders in the European Parliament, and ECB and IMF leaders) from 2007 to 2015. These speeches vary in sentiment, topics and ideology, allowing for fine-grained, over-time comparison of representation in the EU. The member states we included are Czech Republic, France, Germany, Greece, Netherlands, Italy, Spain, United Kingdom, Poland and Portugal.

Schumacher, G, Schoonvelde, M., Dahiya, T., Traber, D, & de Vries, E. (2016): *EUSpeech: a New Dataset of EU Elite Speeches*. [doi:10.7910/DVN/XPCVEI](http://dx.doi.org/10.7910/DVN/XPCVEI)

Download and unpack the following file:
```
speeches_csv.tar.gz
```

I already did this for you. Let's have a look at the files we downlaoded:

In [ ]:
%ls ../class_data/EUSpeech/Cleaned_Speeches/

## Let's start!
Let's retrieve a list of all speeches from one of the files. Of course, we could also loop over all the files...

In [20]:
speeches_nl = pd.read_csv('../class_data/EUSpeech/Cleaned_Speeches/Speeches_NL_Cleaned.csv',header=None)
#To read all files instead
#filelist = glob('../class_data/EUSpeech/Cleaned_Speeches/Speeches_*_Cleaned.csv')
#speeches = pd.concat([pd.read_csv(file,header=None) for file in filelist])
df = speeches_nl.loc[speeches_nl[7]=="en"]
df.head()

,0,1,2,3,4,5,6,7
0,'A roadmap for sustainable recovery',23-09-2010,Netherlands,J.P. Balkenende,1404,"<p>Ladies and gentlemen,</p><p>It is an honour...",ladi gentlemen honour today introduc theme rec...,en
1,Speech by Prime Minister Balkenende in New Yor...,22-09-2010,Netherlands,J.P. Balkenende,840,"<p>Ladies and gentlemen,</p><p>Today we are lo...",ladi gentlemen today look back ten year sinc m...,en
3,Speech at Canadian War Cemetery Bergen op Zoom,06-05-2010,Netherlands,J.P. Balkenende,545,"<p>Prime Minister Harper, Excellencies, honour...",prime minist harper excel honour veteran ladi ...,en
4,Speech by the Prime Minister at the Fudan Univ...,30-04-2010,Netherlands,J.P. Balkenende,2753,"<p>Ni hao pengyoumen, </p><p> Your Excellency,...",ni hao pengyoumen excel professor qin professo...,en
5,Speech at the United Nations Memorial Cemetery...,29-04-2010,Netherlands,J.P. Balkenende,581,"<p>Honoured veterans, ladies and gentlemen,</p...",honour veteran ladi gentlemen today offici fun...,en


In [ ]:
len(speeches_nl)

# 1. A simple, top-down approach to ACA: Let's count how many times a regular expression is mentioned!



Regular Expressions

There are a lot of online tutorials explaining regular expressions (and you can read up in my book or on the slides), so I won't go into detail here how to construct one. But let's look at a prototypical usecase: Counting how often something is mentioned in texts. Let's start by examing one single speech:
In [ ]:

speeches[0]

Then we can get a list with all substrings that match the regexp. And, as with any lists, we can calculate its length!
In [ ]:

re.findall(r"[Ee]conomy|[Ee]conomic",speeches[0])

In [ ]:

len(re.findall(r"[Ee]conomy|[Ee]conomic",speeches[0]))



In [26]:
re.findall(r"[Ee]conomy|[Ee]conomic","Economy is interesting")

['Economy']

In [25]:
df["economy"] = df[5].str.count(r"[Ee]conomy|[Ee]conomic")
df.head()

/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,0,1,2,3,4,5,6,7,economy
0,'A roadmap for sustainable recovery',23-09-2010,Netherlands,J.P. Balkenende,1404,"<p>Ladies and gentlemen,</p><p>It is an honour...",ladi gentlemen honour today introduc theme rec...,en,10
1,Speech by Prime Minister Balkenende in New Yor...,22-09-2010,Netherlands,J.P. Balkenende,840,"<p>Ladies and gentlemen,</p><p>Today we are lo...",ladi gentlemen today look back ten year sinc m...,en,3
3,Speech at Canadian War Cemetery Bergen op Zoom,06-05-2010,Netherlands,J.P. Balkenende,545,"<p>Prime Minister Harper, Excellencies, honour...",prime minist harper excel honour veteran ladi ...,en,0
4,Speech by the Prime Minister at the Fudan Univ...,30-04-2010,Netherlands,J.P. Balkenende,2753,"<p>Ni hao pengyoumen, </p><p> Your Excellency,...",ni hao pengyoumen excel professor qin professo...,en,12
5,Speech at the United Nations Memorial Cemetery...,29-04-2010,Netherlands,J.P. Balkenende,581,"<p>Honoured veterans, ladies and gentlemen,</p...",honour veteran ladi gentlemen today offici fun...,en,0


**Play around! Look wether you can code your own stuff**
If you want to save the results, you can do so with the .to_csv(), .to_excel(), ... methods:

In [28]:
df.to_csv('./data/myoutput.csv')

# 2. A sophisticated, bottom-up approach: LDA topic modelling

# From text to features: preprocessing, tokens, n-grams
## General approach

From a machine-learning perspective, one could argue that all information in a text might be useful information. However, we are interested in getting *interpretable* topics, so even if for instance the use of specific HTML tags would help us distinguising between some documents, we want to get rid of them. More in general, we start by cleaning up a bit to get only 'real' text.

### Some typical clean-up steps:
*[I realize you did not cover list comprehensions in your corse yet, so the commands below may look a bit like weird magic to you. It is basically a way of writing a for loop in one single line. For now, just accept them as they are ;-) ]*

In [53]:
df[5] = df[5].str.lower()
df[5] = df[5].str.replace(r"<p>|</p>"," ") #remove html tags
regex = re.compile('[%s]' % re.escape(punctuation))
df[5] = df[5].str.replace(regex,"") #remove punctuation
df[5] = df[5].str.replace(r" +"," ") #remove double spaces

/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Let's look at the first speech to check everything's fine.

In [56]:
df[5].head()

0     ladies and gentlemen it is an honour to be he...
1     ladies and gentlemen today we are looking bac...
3     prime minister harper excellencies honoured v...
4     ni hao pengyoumen your excellency professor q...
5     honoured veterans ladies and gentlemen today ...
Name: 5, dtype: object

However, as with other unsupervised machine learning techniques, we are not really interesting a long string of text. We rather want to have each document being represented by a set of *features*. To this end, `gensim` has a finciton `doc2bow` that converts a list of words (tokens) to `(token_id, token_count)` tuples. 

## Stopwords
We immediately see that the result of our first LDA doesn't make much sense: We see only stopwords. 


### Explicit stopword removal
The most straightforward approach is to use a pre-existing list with stopwords, possibly with the addition of some own, case-specific words. We then split up each speech in words, and only if a word is not on the stopwordlist, we keep it and join it with the previous and next word using a space.

In [74]:
mystopwords = set(stopwords.words('english')) # use default NLTK stopword list; alternatively:
# mystopwords = set(open('mystopwordfile.txt').readlines())  #read stopword list from a textfile with one stopword per line
df["clean"] = df[5].str.split(" ")
df["clean"] = df["clean"].apply(lambda x: [word for word in x if ((word not in mystopwords) and (len(word)>1))])
df.head()

/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,0,1,2,3,4,5,6,7,economy,clean
0,'A roadmap for sustainable recovery',23-09-2010,Netherlands,J.P. Balkenende,1404,ladies and gentlemen it is an honour to be he...,ladi gentlemen honour today introduc theme rec...,en,10,"[ladies, gentlemen, honour, today, introduce, ..."
1,Speech by Prime Minister Balkenende in New Yor...,22-09-2010,Netherlands,J.P. Balkenende,840,ladies and gentlemen today we are looking bac...,ladi gentlemen today look back ten year sinc m...,en,3,"[ladies, gentlemen, today, looking, back, ten,..."
3,Speech at Canadian War Cemetery Bergen op Zoom,06-05-2010,Netherlands,J.P. Balkenende,545,prime minister harper excellencies honoured v...,prime minist harper excel honour veteran ladi ...,en,0,"[prime, minister, harper, excellencies, honour..."
4,Speech by the Prime Minister at the Fudan Univ...,30-04-2010,Netherlands,J.P. Balkenende,2753,ni hao pengyoumen your excellency professor q...,ni hao pengyoumen excel professor qin professo...,en,12,"[ni, hao, pengyoumen, excellency, professor, q..."
5,Speech at the United Nations Memorial Cemetery...,29-04-2010,Netherlands,J.P. Balkenende,581,honoured veterans ladies and gentlemen today ...,honour veteran ladi gentlemen today offici fun...,en,0,"[honoured, veterans, ladies, gentlemen, today,..."


BUUUT:

### TF-IDF scores instead of word counts as features
Explicitly removing stopwords is a common practice and often very useful. We shouldn't forget, though, that there are some problematic aspects to it as well
- It is kind of arbitrary what is on the stopword list and what now
- Depending on the research question one is interested in, it might differ what words are 'meaningful'
- Although the list is meant to consist of words that occur with a high frequency in all texts, it is not based on actual frequencies in the corpus but set a priori.
A different approach would therefore be to simply use (a) the frequency of each word in the corpus and (b) the number of documents in which the document occurs. 
In other words: If we use tf-idf scores (term frequency weighed by the inverse document frequncy) instead of raw word counts as featues, the stopwords should disappear automatically:

In [75]:
# to avoid messing things up, I'll call all objects relating to our first model _m1",
#this we have done in cla         # convert all strings to list of words
ldainput_m1 = corpora.Dictionary(df["clean"].tolist())                       # assign a token_id to each word
id2word_m1 = corpora.Dictionary(ldainput_m1)                       # assign a token_id to each word
ldacorpus_m1 = [id2word_m1.doc2bow(doc) for doc in ldainput_m1]       # represent each speech by (token_id, token_count) tuples"
tfidfcorpus_m1 = models.TfidfModel(ldacorpus_m1)
lda_m1 = models.ldamodel.LdaModel(corpus=tfidfcorpus_m1[ldacorpus_m1],id2word=id2word_m1,num_topics=10)
lda_m1.print_topics(num_words=5)

2018-01-21 18:19:56,676 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-01-21 18:19:56,754 : INFO : built Dictionary(7383 unique tokens: ['uk', 'awakening', 'adapt', 'days', 'teaches']...) from 132 documents (total 57614 corpus positions)
2018-01-21 18:19:56,755 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


TypeError: 'int' object is not iterable

## Filtering extremes
Just as we don't want to include words that are all over the place and do little to distinguish documents, we also do not want to include words that virtually never occur. If among millions of words, a word occurs exactly one time, it might be simply a spelling mistake. But even if it is not, it does not help us to infer topics across documents. 

Also in purely pragmatic terms, it makes sense to remove unneccessary features to speed up the analysis process.


In [ ]:
id2word_m4 = corpora.Dictionary(ldainput_m1)        # reuse input from M1     

id2word_m4.filter_extremes(no_below=5, no_above=0.5)   # do not consider all words that occur in less than n=5 documents
                                                    # or in more than 50% of all documents.

ldacorpus_m4 = [id2word_m4.doc2bow(doc) for doc in ldainput_m1]
tfidfcorpus_m4 = models.TfidfModel(ldacorpus_m4)
lda_m4 = models.ldamodel.LdaModel(corpus=tfidfcorpus_m4[ldacorpus_m4],id2word=id2word_m4,num_topics=10)
lda_m4.print_topics(num_words=5)

## Other preprocessing ideas

### Stemming
Stemming can be useful to avoid that 'economics', 'economic', and 'economy' are seen as different concepts by the topic model. In practice, however, standard stemming algorithms are far from perfect:

In [ ]:
stemmer = nltk.stem.snowball.EnglishStemmer()
speeches_nl_stemmed = [" ".join([stemmer.stem(word) for word in speech.split()]) for speech in speeches_nl]
speeches_nl_stemmed[0][:500]

### Parsing and retaining only nouns and adjectives
Depending on the specific use case at hand, one might discover that some parts of speech (POS) are more informative than others. We could, for instance, create a topic model based on only the nouns and adjectives in a text, disregarding everything else. 
Look at the NLTK documentation to find out what each code means (e.g., 'NN' is 'noun') 

In [ ]:
speeches_nl_nounsadj=[]
for speech in speeches_nl:
    tokens = nltk.word_tokenize(speech)
    tagged = nltk.pos_tag(tokens)
    cleanspeech = ""
    for element in tagged:
        if element[1] in ('NN','NNP','JJ'):
            cleanspeech=cleanspeech+element[0]+" "
    speeches_nl_nounsadj.append(cleanspeech)

In [ ]:
speeches_nl_nounsadj[0][:500]

### Using ngrams as features
Topic models follow a bag-of-words approach, meaning they do not take word order into account. However, sometimes we want to be able to do so to a limited extend: The "white house" is something else than a "house with a white wall", even though both strings contain the words 'white' and 'house'. We can do so by joining adjacent words together in so-called bigrams (or trigrams, if we take three words).

In [ ]:
speeches_nl_bigrams = [["_".join(tup) for tup in nltk.ngrams(speech.split(),2)] for speech in speeches_nl_clean]

In [ ]:
# maybe we want both unigrams and bigrams in the feature set?
assert len(speeches_nl_clean)==len(speeches_nl_bigrams)
speeches_nl_uniandbigrams = []
for a,b in zip([speech.split() for speech in speeches_nl_clean],speeches_nl_bigrams):
    speeches_nl_uniandbigrams.append(a + b)

In [ ]:
len(speeches_nl_uniandbigrams[6]),len(speeches_nl_bigrams[6]),len(speeches_nl_clean[6].split())

In [ ]:
id2word_m5 = corpora.Dictionary(speeches_nl_uniandbigrams)                       
id2word_m5.filter_extremes(no_below=5, no_above=0.5)
ldacorpus_m5 = [id2word_m5.doc2bow(doc) for doc in speeches_nl_uniandbigrams]
tfidfcorpus_m5 = models.TfidfModel(ldacorpus_m5)
lda_m5 = models.ldamodel.LdaModel(corpus=tfidfcorpus_m5[ldacorpus_m5],id2word=id2word_m5,num_topics=10)
lda_m5.print_topics(num_words=5)

## Summing up
While there are different ways to achieve the desired results and different modules to help you with it (writing from scratch, NLTK, but also gensim.utils), these are some steps to consider when transforming texts to feature sets for topic modeling (recall that not all of them might be neccessary of even diserable, depending on the use case):

- transforming to lowercase
- removing stopwords
- stemming
- POS-tagging (and removing unwanted elements)
- filtering extremely common and extremely uncommon words
- ngrams and/or unigrams as features?
- raw frequencies or TF-IDF scores as features?



# 6. Visualizing and interactively exploring topic models
A great tool for interactively exploring topicmodels is pyLDAvis.
pyLDAvis can estimate its own topic models, but it als has a nice function called `gensim.prepare`, which you can use to visualize the model you already estimated with gensim.

In [ ]:
vis_data = pyLDAvis.gensim.prepare(lda_m5,ldacorpus_m5,id2word_m5)
pyLDAvis.display(vis_data)

# 7. What’s the next step in the pipeline? Using the results of a topic model

Until know, we have mainly considered the interpretation of the topics themselves. While it can indeed be interesting to use topic models to summarize and interpret large corpora, this is usually not where social scientists stop: We want to relate the topics back to documents to say something about which topics occur in which documents.

## Saving topic scores to a file
Somewhat similar to factor analysis and principal component analysis, where one can also store factor scores that indicate how high a specific case scores on each of the factors that were identified, for each document, we can estimate a score for each of the topics we identified.

To do so, we can simply call the `.inference()` method on the model we estimated. 

In [ ]:
scoresperdoc=lda_m5.inference(ldacorpus_m5)

In [ ]:
len(scoresperdoc[0])

We could do a lot of different stuff with the resulting matrix, in which each row represents one of the documents and each row consists of one score for each topic.
For example, we just could create a tab-separated file:

In [ ]:
with open("topicscores.tsv","w",encoding="utf-8") as fo:
    for row in scoresperdoc[0]:
       fo.write("\t".join(["{:0.3f}".format(score) for score in row]))
       fo.write("\n")

... or we put it into a Pandas dataframe

In [ ]:
df = pd.DataFrame.from_records(scoresperdoc[0])
df

We see that these scores are extremely skewed. Maybe we just want to know which topics score really high? Let's recode:

In [ ]:
df2 = df.applymap(lambda x: int(x>10))
df2

Or a heatmap?

In [ ]:
plt.figure(figsize=(10, 20))
sns.heatmap(df)

# Tuning alpha and eta
different parameters. From docstring:
```
`alpha` and `eta` are hyperparameters that affect sparsity of the document-topic
(theta) and topic-word (lambda) distributions. Both default to a symmetric
1.0/num_topics prior.

`alpha` can be set to an explicit array = prior of your choice. It also
support special values of 'asymmetric' and 'auto': the former uses a fixed
normalized asymmetric 1.0/topicno prior, the latter learns an asymmetric
prior directly from your data.

`eta` can be a scalar for a symmetric prior over topic/word
distributions, or a matrix of shape num_topics x num_words, which can
be used to impose asymmetric priors over the word distribution on a
per-topic basis. This may be useful if you want to seed certain topics
with particular words by boosting the priors for those words.  It also
supports the special value 'auto', which learns an asymmetric prior
directly from your data.
```


In [ ]:
# example for different specification: repeat analysis 10 times, while learning alpha and eta from the data 
# instead of using 1/number of topics as defailt
lda_m6 = models.ldamodel.LdaModel(corpus=tfidfcorpus_m5[ldacorpus_m5],id2word=id2word_m5,num_topics=50, alpha='auto', eta = 'symmetric',passes=10)


In [ ]:
vis_data = pyLDAvis.gensim.prepare(lda_m6,ldacorpus_m5,id2word_m5)
pyLDAvis.display(vis_data)




# Exercises

**Choose different input data, for instance the speeches from a different country. Construct a topic model.**

1. Reflect on what choices you made and why:

- Which, if any preprocessing?
- Which features? (e.g., bigram counts; unigrams represented as tf-idf scores; ...)
- Which other model specifications?

2. What did your analysis tell you about the data set?

# How further?

You got interested an want to continue with ACA?

Maybe these ressources are helpful:


- http://github.com/damian0604/bdaca (in particular the book)
- http://damiantrilling.net/tools


In [ ]:
print("FINISHED RUNNING:",str(datetime.now()))